In [214]:
# !pip install -r requirements.txt
# !python -m nltk.downloader stopwords
# !python -m nltk.downloader wordnet
# !python -m nltk.downloader omw-1.4

In [215]:
from nltk.corpus import stopwords
import snscrape.modules.telegram as telegram
import pandas as pd
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score

In [216]:

scraper = telegram.TelegramChannelScraper('thebull_crypto')
telegram_df = pd.DataFrame()
for i,tel in enumerate(scraper.get_items()):
  if ( i < 100):
    telegram_data = [{'Date':tel.date,'Content':tel.content}]
    df = pd.DataFrame(telegram_data)
    telegram_df = pd.concat([telegram_df,df],axis=0)
  else:
    break

telegram_df

,Date,Content
0,2022-07-19 14:54:04+00:00,$JASMY still can buy here!
0,2022-07-19 14:53:04+00:00,$SOL ✅
0,2022-07-19 14:35:38+00:00,$QI major breakout is here!
0,2022-07-19 13:21:45+00:00,$CLV getting ready 👌🏻
0,2022-07-19 13:20:33+00:00,"$OGN is on the way to $0,3 👀🚀"
...,...,...
0,2022-06-24 18:21:39+00:00,With only $30m mcap $TRB will go up +100% easily.
0,2022-06-24 18:10:55+00:00,Bitcoin realized 30d volatility hits 12-month ...
0,2022-06-24 17:38:42+00:00,$JASMY/BTC breakout of 50-ish and it will go u...
0,2022-06-24 17:10:23+00:00,Buying some $TRB on this breakout!


In [217]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
# current_date =date.today()
def text_cleaning(text):
    if(text!=None):
        stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
        return str(normalized.split())

# telegram_df["Cleaned Text"]= telegram_df["Content"]
telegram_df["Cleaned Text"]= telegram_df["Content"].apply(text_cleaning)


In [218]:
telegram_df

,Date,Content,Cleaned Text
0,2022-07-19 14:54:04+00:00,$JASMY still can buy here!,"['jasmy', 'still', 'buy', 'here']"
0,2022-07-19 14:53:04+00:00,$SOL ✅,"['sol', '✅']"
0,2022-07-19 14:35:38+00:00,$QI major breakout is here!,"['qi', 'major', 'breakout', 'here']"
0,2022-07-19 13:21:45+00:00,$CLV getting ready 👌🏻,"['clv', 'getting', 'ready', '👌🏻']"
0,2022-07-19 13:20:33+00:00,"$OGN is on the way to $0,3 👀🚀","['ogn', 'way', '03', '👀🚀']"
...,...,...,...
0,2022-06-24 18:21:39+00:00,With only $30m mcap $TRB will go up +100% easily.,"['30m', 'mcap', 'trb', 'go', '100', 'easily']"
0,2022-06-24 18:10:55+00:00,Bitcoin realized 30d volatility hits 12-month ...,"['bitcoin', 'realized', '30d', 'volatility', '..."
0,2022-06-24 17:38:42+00:00,$JASMY/BTC breakout of 50-ish and it will go u...,"['jasmybtc', 'breakout', '50ish', 'go', 'hard']"
0,2022-06-24 17:10:23+00:00,Buying some $TRB on this breakout!,"['buying', 'trb', 'breakout']"


In [219]:
df = pd.read_csv("Prediction - Prediction.csv")
X = telegram_df.loc[:,"Cleaned Text"]
Y = df.iloc[:,2]
Y = Y.apply(lambda x : str(x))
Y

0     1
1     1
2     1
3     1
4     2
     ..
95    1
96    2
97    1
98    2
99    1
Name: Prediction, Length: 100, dtype: object

In [220]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=.2)
model = make_pipeline(TfidfVectorizer(lowercase=False), MultinomialNB())
model.fit(X_train, Y_train)
pred = model.predict(X_test)
model.score(pred,Y_test)
Y_train

TypeError: expected string or bytes-like object

In [ ]:
def my_predictions(my_sentence, model):
    prediction = model.predict([my_sentence])
    return prediction

result=list()
# telegram_df["Cleaned Text"].apply(lambda x: x.lower())
for i in telegram_df["Cleaned Text"]:
    result.append(my_predictions(str(i),model))
telegram_df["Result"]=result
telegram_df

,Date,Content,Cleaned Text,Result
0,2022-07-19 14:54:04+00:00,$JASMY still can buy here!,$JASMY still can buy here!,[1]
0,2022-07-19 14:53:04+00:00,$SOL ✅,$SOL ✅,[1]
0,2022-07-19 14:35:38+00:00,$QI major breakout is here!,$QI major breakout is here!,[1]
0,2022-07-19 13:21:45+00:00,$CLV getting ready 👌🏻,$CLV getting ready 👌🏻,[1]
0,2022-07-19 13:20:33+00:00,"$OGN is on the way to $0,3 👀🚀","$OGN is on the way to $0,3 👀🚀",[1]
...,...,...,...,...
0,2022-06-01 17:34:50+00:00,OpenSea employee charged with insider trading ...,OpenSea employee charged with insider trading ...,[2]
0,2022-06-01 10:59:46+00:00,Binance raises $500 million fund to invest in ...,Binance raises $500 million fund to invest in ...,[1]
0,2022-06-01 07:28:48+00:00,Rebuying $MASK 👀,Rebuying $MASK 👀,[2]
0,2022-05-31 18:51:28+00:00,Bitcoin’s correlation with U.S. equities weakens.,Bitcoin’s correlation with U.S. equities weakens.,[2]


In [ ]:
telegram_df.to_csv("SD.csv")

In [ ]:
!pip freeze > requirements.txt